# Müzik Türü Sınıflandırma Projesi

Bu notebook, FMA (Free Music Archive) veri setini kullanarak müzik türü sınıflandırma modeli geliştirmek için veri hazırlama ve dengeleme işlemlerini içermektedir.

## Gerekli Kütüphanelerin İçe Aktarılması
Aşağıdaki hücrede, projede kullanılacak temel Python kütüphaneleri import edilmektedir:

In [ ]:
# Core libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter

# Machine learning libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE

# PyTorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Configuration
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set(style='whitegrid')
print("📚 All libraries imported successfully!")

## Yardımcı Fonksiyonlar

### Sınıf Dağılımı Görselleştirme Fonksiyonu
Aşağıdaki fonksiyon, veri setindeki sınıf dağılımlarını görselleştirmek için kullanılacaktır. Bu görselleştirme, veri dengesizliğini anlamamıza yardımcı olur.

In [ ]:
def plot_class_distribution(y, class_names, title):
    """Plot class distribution with improved visualization"""
    counts = pd.Series(y).value_counts().sort_index()
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(range(len(counts)), counts.values, color='skyblue', alpha=0.7)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Music Genre')
    plt.ylabel('Number of Samples')
    plt.xticks(range(len(counts)), [class_names[i] for i in counts.index], rotation=45, ha='right')
    
    # Add value labels on bars
    for bar, count in zip(bars, counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                str(count), ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"Total samples: {sum(counts.values):,}")
    print(f"Number of classes: {len(counts)}")
    print(f"Min samples per class: {min(counts.values)}")
    print(f"Max samples per class: {max(counts.values)}")
    print(f"Average samples per class: {np.mean(counts.values):.1f}")

## Veri Yükleme ve Ön İşleme

Bu bölümdeki fonksiyon:
- FMA metadata dosyalarını yükler
- Gerekli sütunları seçer
- Eksik verileri temizler
- Etiketleri kodlar
- Veriyi sayısal formata dönüştürür

In [ ]:
def load_fma_data(data_dir='fma_metadata'):
    """Load and preprocess FMA dataset"""
    tracks_path = os.path.join(data_dir, 'tracks.csv')
    features_path = os.path.join(data_dir, 'features.csv')
    
    # Check if files exist
    if not os.path.exists(tracks_path) or not os.path.exists(features_path):
        raise FileNotFoundError(f"Required files not found in {data_dir}/")
    
    print("📂 Loading FMA metadata files...")
    tracks = pd.read_csv(tracks_path, index_col=0, header=[0,1])
    features = pd.read_csv(features_path, index_col=0, header=[0,1])
    
    print(f"✅ Loaded tracks: {tracks.shape}, features: {features.shape}")
    
    # Remove statistics columns (non-audio features)
    audio_features = features.loc[:, features.columns.get_level_values(0) != 'statistics']
    print(f"📊 Audio features after removing statistics: {audio_features.shape[1]} features")
    
    # Get genre labels
    genre_series = tracks[('track', 'genre_top')].dropna()
    common_index = audio_features.index.intersection(genre_series.index)
    
    # Extract final data
    X = audio_features.loc[common_index]
    y_labels = genre_series.loc[common_index]
    
    # Clean and preprocess
    X = X.fillna(0).replace([np.inf, -np.inf], 0).astype(np.float32)
    X.index = X.index.astype(str)
    
    # Encode labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_labels)
    
    print(f"\n🎯 Final dataset:")
    print(f"   Features: {X.shape[1]}")
    print(f"   Samples: {X.shape[0]:,}")
    print(f"   Classes: {len(label_encoder.classes_)}")
    print(f"   Genres: {', '.join(label_encoder.classes_)}")
    
    return X, y, label_encoder

## Başlangıç Veri Analizi

Verinin ilk yüklemesini yapıp, başlangıçtaki sınıf dağılımını inceleyelim. Bu analiz, veri dengesizliği problemini görselleştirmemize yardımcı olacak.

In [ ]:
# Load and analyze the dataset
X, y, label_encoder = load_fma_data()

# Visualize initial class distribution
plot_class_distribution(y, label_encoder.classes_, 'Initial Class Distribution')

## Veri Bölme ve Eğitim Seti Analizi

Veriyi eğitim ve test setlerine ayırıp, eğitim setindeki sınıf dağılımını inceliyoruz. Stratified split kullanarak orijinal dağılımı koruyoruz.

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"📊 Data split completed:")
print(f"   Training set: {X_train.shape}")
print(f"   Test set: {X_test.shape}")

# Visualize training set distribution
plot_class_distribution(y_train, label_encoder.classes_, 'Training Set Distribution')

## Veri Dengeleme - Aşama 1

İlk aşamada, çok az örneğe sahip sınıflar için RandomOverSampler kullanılıyor. Bu aşama, BorderlineSMOTE için yeterli örnek sayısına ulaşmamızı sağlar.

In [ ]:
def balance_dataset(X_train, y_train, label_encoder):
    """Balance the dataset using two-phase approach"""
    print("⚖️ Phase 1: Random Oversampling for minority classes...")
    
    # Phase 1: Random oversampling for very small classes
    min_threshold = 60
    class_counts = Counter(y_train)
    small_classes = {cls: min_threshold for cls, count in class_counts.items() if count < min_threshold}
    
    if small_classes:
        ros = RandomOverSampler(sampling_strategy=small_classes, random_state=42)
        X_partial, y_partial = ros.fit_resample(X_train, y_train)
        print(f"   ✅ Oversampled {len(small_classes)} minority classes")
    else:
        X_partial, y_partial = X_train, y_train
        print(f"   ℹ️ No minority classes found below threshold {min_threshold}")
    
    print("\n⚖️ Phase 2: BorderlineSMOTE for remaining imbalance...")
    try:
        smote = BorderlineSMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X_partial, y_partial)
        print(f"   ✅ SMOTE completed successfully")
    except Exception as e:
        print(f"   ⚠️ SMOTE failed: {e}. Using partial balanced data.")
        X_balanced, y_balanced = X_partial, y_partial
    
    print(f"\n📈 Balancing results: {X_train.shape} → {X_balanced.shape}")
    return X_balanced, y_balanced

# Apply balancing
X_balanced, y_balanced = balance_dataset(X_train, y_train, label_encoder)

# Visualize balanced distribution
plot_class_distribution(y_balanced, label_encoder.classes_, 'Balanced Training Set')

## Gelişmiş Özellik Seçimi (LSTM İçin Optimize)

Model performansını artırmak ve LSTM için optimize edilmiş özellik seçimi uygulayacağız. Mutual Information ile temporal dependencies yakalarken, correlation filtering ile multicollinearity'yi azaltacağız.

In [ ]:
# Gelişmiş Özellik Seçimi - LSTM için Optimize
from sklearn.feature_selection import mutual_info_classif, RFE, SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

def select_features(X, y, n_features=120):
    """Select best features using SelectKBest with F-test"""
    print(f"🔍 Selecting top {n_features} features from {X.shape[1]} total features...")
    
    # Convert to numpy if needed
    if hasattr(X, 'values'):
        X_array = X.values
    else:
        X_array = X
    
    # Apply SelectKBest
    selector = SelectKBest(score_func=f_classif, k=min(n_features, X_array.shape[1]))
    X_selected = selector.fit_transform(X_array, y)
    
    # Get feature information
    selected_mask = selector.get_support()
    selected_indices = np.where(selected_mask)[0]
    feature_scores = selector.scores_[selected_mask]
    
    print(f"✅ Selected {X_selected.shape[1]} features")
    print(f"📊 Score range: {feature_scores.min():.2f} - {feature_scores.max():.2f}")
    print(f"📊 Average score: {feature_scores.mean():.2f}")
    
    return X_selected, selected_indices

# Apply feature selection
X_train_selected, selected_indices = select_features(X_balanced, y_balanced)
X_test_selected = X_test.iloc[:, selected_indices].values if hasattr(X_test, 'iloc') else X_test[:, selected_indices]

print(f"\n🎯 Feature selection completed:")
print(f"   Training features: {X_train_selected.shape}")
print(f"   Test features: {X_test_selected.shape}")

# Skip the complex feature selection and use the simple approach above
# Original complex code follows but is commented out for reference:
"""
print('\nLSTM için optimize edilmiş gelişmiş özellik seçimi uygulanıyor...')

# Toplam seçilecek özellik sayısı
total_k = 160  # 4 kategoriye bölünecek (MFCC, Chroma, Spectral, Others)
k_per_category = total_k // 4  # Her kategori için eşit sayıda özellik

print(f"Toplam özellik sayısı: {X_balanced.shape[1]}, Seçilecek özellik sayısı: {total_k}")
print(f"Her özellik kategorisinden seçilecek: {k_per_category}")

# Özellik isimlerini oluştur
if hasattr(X_balanced, 'columns'):
    if isinstance(X_res.columns, pd.MultiIndex):
        feature_names = [f"{col[0]}_{col[1]}" if len(col) > 1 else str(col[0]) for col in X_res.columns]
    else:
        feature_names = [str(col) for col in X_res.columns]
else:
    feature_names = [f'feature_{i}' for i in range(X_res.shape[1])]

print(f"Özellik isimleri oluşturuldu: {len(feature_names)} adet")

# X_res'i numpy array'e dönüştür
X_res_array = X_res.values
X_test_array = X_test.values

# Hızlı ve Optimize Edilmiş Özellik Seçimi (LSTM için)
print("\n⚡ Hızlı ve Optimize Edilmiş özellik seçimi uygulanıyor...")
print("   ✅ SelectKBest + Basit korelasyon filtreleme")
print("   ✅ Çok daha hızlı ve etkili")
print("   ✅ LSTM için yeterince iyi performans")

# Özellik kategorilerini bul
mfcc_indices = [i for i, name in enumerate(feature_names) if 'mfcc' in name.lower()]
chroma_indices = [i for i, name in enumerate(feature_names) if 'chroma' in name.lower()]
spectral_indices = [i for i, name in enumerate(feature_names) if any(spec in name.lower() for spec in ['spectral', 'centroid', 'bandwidth', 'contrast', 'rolloff'])]
other_indices = [i for i in range(len(feature_names)) 
                if i not in mfcc_indices and i not in chroma_indices and i not in spectral_indices]

print(f"\nÖzellik kategori sayıları:")
print(f"- MFCC: {len(mfcc_indices)} özellik")
print(f"- Chroma: {len(chroma_indices)} özellik")
print(f"- Spectral: {len(spectral_indices)} özellik")
print(f"- Diğer: {len(other_indices)} özellik")

# 1. Adım: Her kategoriden SelectKBest ile hızlı seçim
selected_indices = []
category_info = []

for category_name, indices in [('MFCC', mfcc_indices), ('Chroma', chroma_indices), 
                              ('Spectral', spectral_indices), ('Others', other_indices)]:
    if len(indices) > 0:
        X_category = X_res_array[:, indices]
        
        # SelectKBest ile hızlı seçim (F-test)
        k_category = min(k_per_category, len(indices))
        selector = SelectKBest(score_func=f_classif, k=k_category)
        selector.fit(X_category, y_res)
        
        # Seçilen özelliklerin orijinal indekslerini al
        selected_mask = selector.get_support()
        selected_category_indices = [indices[i] for i in range(len(indices)) if selected_mask[i]]
        selected_indices.extend(selected_category_indices)
        
        category_info.append({
            'name': category_name,
            'total': len(indices),
            'selected': len(selected_category_indices),
            'avg_score': np.mean(selector.scores_[selected_mask])
        })
        
        print(f"   {category_name}: {len(selected_category_indices)} özellik seçildi, Ortalama F-score: {np.mean(selector.scores_[selected_mask]):.2f}")

# 2. Adım: Basit korelasyon filtreleme (çok daha hızlı)
print(f"\n🔍 Basit korelasyon filtreleme uygulanıyor...")
final_indices = selected_indices.copy()

if len(final_indices) > 50:  # Sadece çok fazla özellik varsa korelasyon filtrele
    print("   Korelasyon matrisi hesaplanıyor...")
    corr_matrix = np.corrcoef(X_res_array[:, final_indices].T)
    
    # Basit korelasyon filtreleme (ilk bulduğunu çıkar)
    to_remove = set()
    for i in range(len(corr_matrix)):
        if i in to_remove:
            continue
        for j in range(i+1, len(corr_matrix)):
            if j not in to_remove and abs(corr_matrix[i, j]) > 0.95:  # Çok yüksek korelasyon
                to_remove.add(j)  # j'yi çıkar (basit kural)
                if len(to_remove) > 10:  # Maksimum 10 özellik çıkar
                    break
        if len(to_remove) > 10:
            break
    
    # Çıkarılacak indeksleri kaldır
    final_indices = [final_indices[i] for i in range(len(final_indices)) if i not in to_remove]
    
    print(f"   {len(to_remove)} yüksek korelasyonlu özellik çıkarıldı")
    print(f"   Final özellik sayısı: {len(final_indices)}")
else:
    print("   Özellik sayısı az, korelasyon filtreleme atlanıyor")

hybrid_indices = final_indices

# Seçilen özellikleri uygula
X_res_selected = X_res_array[:, hybrid_indices]
X_test_selected = X_test_array[:, hybrid_indices]
selected_feature_names = [feature_names[i] for i in hybrid_indices]

print(f"\n✅ Hızlı Özellik Seçimi tamamlandı!")
print(f"📊 Final özellik sayısı: {len(hybrid_indices)}")
print(f"📋 Seçilen özelliklerin boyutu: {X_res_selected.shape}")

# Final kategori dağılımını kontrol et
final_mfcc = len([name for name in selected_feature_names if 'mfcc' in name.lower()])
final_chroma = len([name for name in selected_feature_names if 'chroma' in name.lower()])
final_spectral = len([name for name in selected_feature_names if any(spec in name.lower() for spec in ['spectral', 'centroid', 'bandwidth', 'contrast', 'rolloff'])])
final_others = len(selected_feature_names) - final_mfcc - final_chroma - final_spectral

print(f"\n📈 Final Kategori Dağılımı:")
print(f"   - MFCC: {final_mfcc} özellik")
print(f"   - Chroma: {final_chroma} özellik")
print(f"   - Spectral: {final_spectral} özellik")
print(f"   - Others: {final_others} özellik")

# Global değişkenleri güncelle
X_res = X_res_selected
X_test = X_test_selected
feature_names = selected_feature_names

print(f"\n🎯 Hızlı özellik seçimi tamamlandı!")
print(f"⚡ Veri şekli güncellendi: X_res {X_res.shape}, X_test {X_test.shape}")
print(f"✨ SelectKBest + Basit korelasyon filtreleme kullanıldı!")

*-----------------------------------------------------------------------------------*
# PyTorch LSTM MODEL EĞİTİMİ
*-----------------------------------------------------------------------------------*

In [ ]:
def prepare_training_data(X_train, X_test, y_train, y_test, test_size=0.1):
    """Prepare and scale data for training"""
    # Split training data into train/validation
    X_train_final, X_val, y_train_final, y_val = train_test_split(
        X_train, y_train, test_size=test_size, stratify=y_train, random_state=42
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_final)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"📊 Data preparation completed:")
    print(f"   Training: {X_train_scaled.shape}")
    print(f"   Validation: {X_val_scaled.shape}")
    print(f"   Test: {X_test_scaled.shape}")
    
    return X_train_scaled, X_val_scaled, X_test_scaled, y_train_final, y_val, scaler

# Prepare data
X_train_scaled, X_val_scaled, X_test_scaled, y_train_final, y_val, scaler = prepare_training_data(
    X_train_selected, X_test_selected, y_balanced, y_test
)

In [ ]:
def create_sequences(X, y, sequence_length=10):
    """Create sequence data for LSTM from features"""
    n_samples, n_features = X.shape
    features_per_step = max(1, n_features // sequence_length)
    
    # Adjust sequence length if needed
    actual_seq_length = min(sequence_length, n_features)
    
    # Create sequences
    X_seq = np.zeros((n_samples, actual_seq_length, features_per_step))
    
    for i in range(n_samples):
        for t in range(actual_seq_length):
            start_idx = t * features_per_step
            end_idx = min(start_idx + features_per_step, n_features)
            if start_idx < n_features:
                X_seq[i, t, :end_idx-start_idx] = X[i, start_idx:end_idx]
    
    return torch.FloatTensor(X_seq), torch.LongTensor(y)

def create_data_loaders(X_train, X_val, X_test, y_train, y_val, y_test, batch_size=128):
    """Create PyTorch data loaders"""
    # Create sequences
    X_train_seq, y_train_tensor = create_sequences(X_train, y_train)
    X_val_seq, y_val_tensor = create_sequences(X_val, y_val)
    X_test_seq, y_test_tensor = create_sequences(X_test, y_test)
    
    print(f"🔄 Sequence shapes:")
    print(f"   Train: {X_train_seq.shape}")
    print(f"   Validation: {X_val_seq.shape}")
    print(f"   Test: {X_test_seq.shape}")
    
    # Create datasets and loaders
    train_dataset = TensorDataset(X_train_seq, y_train_tensor)
    val_dataset = TensorDataset(X_val_seq, y_val_tensor)
    test_dataset = TensorDataset(X_test_seq, y_test_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, X_train_seq.shape

# Create data loaders
train_loader, val_loader, test_loader, input_shape = create_data_loaders(
    X_train_scaled, X_val_scaled, X_test_scaled, y_train_final, y_val, y_test
)

def train_model(model, train_loader, val_loader, device, num_epochs=20, lr=0.001):
    """Train the LSTM model with simplified training loop"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    best_val_acc = 0
    patience_counter = 0
    
    print(f"🚀 Starting training for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate metrics
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        train_acc = train_correct / train_total
        val_acc = val_correct / val_total
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        # Learning rate scheduling
        scheduler.step(val_loss)
        
        # Early stopping check
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1
        
        print(f"Epoch {epoch+1:2d}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        # Early stopping
        if patience_counter >= 5:
            print(f"\n🛑 Early stopping at epoch {epoch+1}")
            break
    
    return train_losses, val_losses, train_accs, val_accs

def evaluate_model(model, test_loader, device, label_encoder):
    """Evaluate model on test set"""
    model.eval()
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    
    print(f"\n🎯 Test Accuracy: {accuracy:.4f}")
    print(f"\n📋 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=label_encoder.classes_, 
                yticklabels=label_encoder.classes_)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return accuracy, y_true, y_pred

def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    """Plot training history"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss plot
    ax1.plot(train_losses, label='Training', marker='o')
    ax1.plot(val_losses, label='Validation', marker='s')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy plot
    ax2.plot(train_accs, label='Training', marker='o')
    ax2.plot(val_accs, label='Validation', marker='s')
    ax2.set_title('Model Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Train the model
print("\n🎵 Training LSTM model...")
train_losses, val_losses, train_accs, val_accs = train_model(
    model, train_loader, val_loader, device, num_epochs=30, lr=0.001
)

# Plot training history
plot_training_history(train_losses, val_losses, train_accs, val_accs)

# Evaluate on test set
test_accuracy, y_true, y_pred = evaluate_model(model, test_loader, device, label_encoder)

print(f"\n✅ Training completed! Final test accuracy: {test_accuracy:.4f}")

In [ ]:
# Attention mekanizması sınıfı
class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.hidden_size = hidden_size
        
        # Attention ağırlıklarını hesaplamak için linear katmanlar
        self.attention_linear = nn.Linear(hidden_size, hidden_size)
        self.context_vector = nn.Linear(hidden_size, 1, bias=False)
        
    def forward(self, lstm_outputs):
        # lstm_outputs şekli: (batch_size, sequence_length, hidden_size)
        
        # Her zaman adımı için attention skorları hesapla
        attention_weights = self.attention_linear(lstm_outputs)  # (batch_size, seq_len, hidden_size)
        attention_weights = torch.tanh(attention_weights)
        attention_scores = self.context_vector(attention_weights)  # (batch_size, seq_len, 1)
        attention_scores = attention_scores.squeeze(2)  # (batch_size, seq_len)
        
        # Softmax ile normalize et
        attention_weights = torch.softmax(attention_scores, dim=1)  # (batch_size, seq_len)
        
        # Weighted sum hesapla
        # attention_weights: (batch_size, seq_len) -> (batch_size, seq_len, 1)
        attention_weights = attention_weights.unsqueeze(2)
        
        # Weighted combination of LSTM outputs
        attended_output = torch.sum(lstm_outputs * attention_weights, dim=1)  # (batch_size, hidden_size)
        
        return attended_output, attention_weights.squeeze(2)

# Bidirectional LSTM model sınıfını tanımlama (Attention ile)
class MusicGenreLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3, bidirectional=True):
        super(MusicGenreLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        # Bidirectional LSTM katmanları
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        
        # Bidirectional LSTM çıkış boyutunu hesapla
        lstm_output_size = hidden_size * 2 if bidirectional else hidden_size
        
        # Attention katmanı (bidirectional output için)
        self.attention = AttentionLayer(lstm_output_size)
        
        # Batch normalization
        self.batch_norm = nn.BatchNorm1d(lstm_output_size)
        
        # Dropout katmanı
        self.dropout = nn.Dropout(dropout)
        
        # Tam bağlantılı katmanlar
        self.fc1 = nn.Linear(lstm_output_size, 128)  # İlk tam bağlantılı katman
        self.fc2 = nn.Linear(128, num_classes)
        
        # Aktivasyon fonksiyonları
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Bidirectional LSTM katmanından geçirme
        # x şekli: (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        
        # lstm_out şekli: (batch_size, sequence_length, hidden_size * 2) for bidirectional
        # veya (batch_size, sequence_length, hidden_size) for unidirectional
        
        # Attention mekanizması uygula
        attended_output, attention_weights = self.attention(lstm_out)
        
        # Batch normalization
        batch_norm_out = self.batch_norm(attended_output)
        
        # İlk tam bağlantılı katman
        fc1_out = self.fc1(batch_norm_out)
        fc1_out = self.relu(fc1_out)
        fc1_out = self.dropout(fc1_out)
        
        # İkinci tam bağlantılı katman (çıkış katmanı)
        out = self.fc2(fc1_out)
        
        return out

# Learning Rate Warmup Scheduler
class WarmupScheduler:
    def __init__(self, optimizer, warmup_epochs, max_lr, min_lr=1e-7):
        self.optimizer = optimizer
        self.warmup_epochs = warmup_epochs
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.current_epoch = 0
        
    def step(self):
        if self.current_epoch < self.warmup_epochs:
            # Linear warmup
            lr = self.min_lr + (self.max_lr - self.min_lr) * (self.current_epoch / self.warmup_epochs)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = lr
        self.current_epoch += 1
        
    def get_lr(self):
        return self.optimizer.param_groups[0]['lr']

# Create simplified LSTM model
class SimpleLSTMClassifier(nn.Module):
    """Simplified LSTM model for music genre classification"""
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(SimpleLSTMClassifier, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Calculate LSTM output size (bidirectional doubles the hidden size)
        lstm_output_size = hidden_size * 2
        
        # Classification layers
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(lstm_output_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )
        
    def forward(self, x):
        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)
        
        # Use the last output for classification
        last_output = lstm_out[:, -1, :]
        
        # Classification
        output = self.classifier(last_output)
        return output

def create_model(input_shape, num_classes, hidden_size=64, num_layers=2, dropout=0.3):
    """Create and initialize the LSTM model"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = SimpleLSTMClassifier(
        input_size=input_shape[2],  # features per timestep
        hidden_size=hidden_size,
        num_layers=num_layers,
        num_classes=num_classes,
        dropout=dropout
    ).to(device)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"🤖 Model created on {device}")
    print(f"📊 Parameters: {trainable_params:,} trainable / {total_params:,} total")
    
    return model, device

# Create model
model, device = create_model(input_shape, len(label_encoder.classes_))
print(f"\n🏗️ Model architecture:")
print(model)

def train_model(model, train_loader, val_loader, device, num_epochs=20, lr=0.001):
    """Train the LSTM model with simplified training loop"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    best_val_acc = 0
    patience_counter = 0
    
    print(f"🚀 Starting training for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate metrics
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        train_acc = train_correct / train_total
        val_acc = val_correct / val_total
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        # Learning rate scheduling
        scheduler.step(val_loss)
        
        # Early stopping check
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1
        
        print(f"Epoch {epoch+1:2d}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        # Early stopping
        if patience_counter >= 5:
            print(f"\n🛑 Early stopping at epoch {epoch+1}")
            break
    
    return train_losses, val_losses, train_accs, val_accs

def evaluate_model(model, test_loader, device, label_encoder):
    """Evaluate model on test set"""
    model.eval()
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    
    print(f"\n🎯 Test Accuracy: {accuracy:.4f}")
    print(f"\n📋 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=label_encoder.classes_, 
                yticklabels=label_encoder.classes_)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return accuracy, y_true, y_pred

def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    """Plot training history"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss plot
    ax1.plot(train_losses, label='Training', marker='o')
    ax1.plot(val_losses, label='Validation', marker='s')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy plot
    ax2.plot(train_accs, label='Training', marker='o')
    ax2.plot(val_accs, label='Validation', marker='s')
    ax2.set_title('Model Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Train the model
print("\n🎵 Training LSTM model...")
train_losses, val_losses, train_accs, val_accs = train_model(
    model, train_loader, val_loader, device, num_epochs=30, lr=0.001
)

# Plot training history
plot_training_history(train_losses, val_losses, train_accs, val_accs)

# Evaluate on test set
test_accuracy, y_true, y_pred = evaluate_model(model, test_loader, device, label_encoder)

print(f"\n✅ Training completed! Final test accuracy: {test_accuracy:.4f}")

In [ ]:
# Eğitim sonuçlarını görselleştirme
def plot_training_history(train_losses, val_losses, train_accs, val_accs, learning_rates=None):
    if learning_rates is not None:
        plt.figure(figsize=(18, 6))
        
        # Kayıp grafiği
        plt.subplot(1, 3, 1)
        plt.plot(train_losses, label='Eğitim', marker='o', alpha=0.7)
        plt.plot(val_losses, label='Doğrulama', marker='*', alpha=0.7)
        plt.title('Model Kaybı')
        plt.xlabel('Epoch')
        plt.ylabel('Kayıp (Cross-Entropy)')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        
        # Doğruluk grafiği
        plt.subplot(1, 3, 2)
        plt.plot(train_accs, label='Eğitim', marker='o', alpha=0.7)
        plt.plot(val_accs, label='Doğrulama', marker='*', alpha=0.7)
        plt.title('Model Doğruluğu')
        plt.xlabel('Epoch')
        plt.ylabel('Doğruluk')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        
        # Learning Rate grafiği
        plt.subplot(1, 3, 3)
        plt.plot(learning_rates, marker='s', alpha=0.7, color='red')
        plt.title('Learning Rate Değişimi')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.yscale('log')  # Log scale for better visualization
        plt.grid(True, linestyle='--', alpha=0.6)
    else:
        plt.figure(figsize=(14, 5))
        
        # Kayıp grafiği
        plt.subplot(1, 2, 1)
        plt.plot(train_losses, label='Eğitim', marker='o')
        plt.plot(val_losses, label='Doğrulama', marker='*')
        plt.title('Model Kaybı')
        plt.xlabel('Epoch')
        plt.ylabel('Kayıp (Cross-Entropy)')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        
        # Doğruluk grafiği
        plt.subplot(1, 2, 2)
        plt.plot(train_accs, label='Eğitim', marker='o')
        plt.plot(val_accs, label='Doğrulama', marker='*')
        plt.title('Model Doğruluğu')
        plt.xlabel('Epoch')
        plt.ylabel('Doğruluk')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# Eğitim sonuçlarını görselleştir
try:
    if 'learning_rates' in locals():
        plot_training_history(train_losses, val_losses, train_accs, val_accs, learning_rates)
        
        # Detaylı learning rate analizi
        print(f"\n📊 Learning Rate İstatistikleri:")
        print(f"Başlangıç LR: {learning_rates[0]:.2e}")
        print(f"Maksimum LR: {max(learning_rates):.2e}")
        print(f"Son LR: {learning_rates[-1]:.2e}")
        print(f"Ortalama LR: {np.mean(learning_rates):.2e}")
    else:
        plot_training_history(train_losses, val_losses, train_accs, val_accs)
except NameError:
    print("Eğitim geçmişi bulunamadı. Önce modeli eğitin.")

# Test veri seti üzerinde değerlendirme
def evaluate_model(model, test_loader, device):
    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    # Doğruluk hesapla
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    
    # Sonuçları yazdır
    print(f"\n🎯 Test Doğruluğu: {accuracy:.4f}")
    
    # Sınıflandırma raporu
    print("\n📋 Sınıflandırma Raporu:")
    print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))
    
    # Karmaşıklık matrisi
    plt.figure(figsize=(12, 10))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title('Karmaşıklık Matrisi')
    plt.xlabel('Tahmin Edilen Etiketler')
    plt.ylabel('Gerçek Etiketler')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return y_true, y_pred

# Test veri seti üzerinde değerlendir
try:
    y_true, y_pred = evaluate_model(model, test_loader, device)
except NameError:
    print("Model bulunamadı. Önce modeli eğitin.")

## Summary

This notebook provides a streamlined approach to music genre classification using the FMA dataset:

1. **Data Loading**: Load and preprocess FMA tracks and features
2. **Data Balancing**: Handle class imbalance using oversampling techniques
3. **Feature Selection**: Select the most informative features for classification
4. **Model Training**: Train a bidirectional LSTM with simplified architecture
5. **Evaluation**: Assess model performance on test data

The refactored code is more modular, maintainable, and easier to understand while maintaining the core functionality.